In [3]:
# -*- coding: utf-8 -*-
# VGG模型，GPU模式

import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter

# from model import *
# 准备数据集
from torch import nn
from torch.utils.data import DataLoader

train_data = torchvision.datasets.CIFAR10(root="./dataset", train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10(root="./dataset", train=False, transform=torchvision.transforms.ToTensor(),
                                         download=True)

# length 长度
train_data_size = len(train_data)
test_data_size = len(test_data)
# 如果train_data_size=10, 训练数据集的长度为：10
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))


# 利用 DataLoader 来加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 创建网络模型


class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x


vgg = VGG()
if torch.cuda.is_available():
    vgg = vgg.cuda()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()
# 优化器
# learning_rate = 0.01
# 1e-2=1 x (10)^(-2) = 1 /100 = 0.01
learning_rate = 1e-3
optimizer = torch.optim.SGD(vgg.parameters(), lr=learning_rate)

# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 20

# 添加tensorboard
writer = SummaryWriter("./logs_train")

for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i+1))

    # 训练步骤开始
    vgg.train()
    for data in train_dataloader:
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
        outputs = vgg(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试步骤开始
    vgg.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
            outputs = vgg(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy /
                      test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(
        vgg, "/home/tzr/Documents/GitHubSYNC/PythonandMLLearning/Pytorch/demo1/vggmodel/vgg_{}.pth".format(i))
    print("模型已保存")

writer.close()


Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000
-------第 1 轮训练开始-------
训练次数：100, Loss: 2.296464681625366
训练次数：200, Loss: 2.3079659938812256
训练次数：300, Loss: 2.3097941875457764
训练次数：400, Loss: 2.305337429046631
训练次数：500, Loss: 2.303525447845459
训练次数：600, Loss: 2.292511224746704
训练次数：700, Loss: 2.284759521484375
整体测试集上的Loss: 360.8021731376648
整体测试集上的正确率: 0.15969999134540558
模型已保存
-------第 2 轮训练开始-------
训练次数：800, Loss: 2.289879322052002
训练次数：900, Loss: 2.2989988327026367
训练次数：1000, Loss: 2.3018925189971924
训练次数：1100, Loss: 2.310460090637207
训练次数：1200, Loss: 2.2976317405700684
训练次数：1300, Loss: 2.2931318283081055
训练次数：1400, Loss: 2.2955543994903564
训练次数：1500, Loss: 2.3041059970855713
整体测试集上的Loss: 359.56475138664246
整体测试集上的正确率: 0.12109999358654022
模型已保存
-------第 3 轮训练开始-------
训练次数：1600, Loss: 2.296522378921509
训练次数：1700, Loss: 2.282158851623535
训练次数：1800, Loss: 2.2938880920410156
训练次数：1900, Loss: 2.276038885116577
训练次数：2000, Loss: